In [4]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
import pymorphy2
import re
import string
from tqdm import tqdm
from collections import Counter
import numpy as np
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from datetime import datetime

tqdm.pandas()
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

from multiprocessing import Pool
morph = pymorphy2.MorphAnalyzer()

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mchelushkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Подготовка данных

In [5]:
commentsDF = pd.read_csv('data/commentsRAW.csv', usecols=['id', 'date', 'text'])
answersDF = pd.read_csv('data/answersRAW.csv', usecols=['id', 'date', 'text'])
commentsDF = commentsDF[commentsDF.text.notna()]
answersDF = answersDF[answersDF.text.notna()]

In [6]:
allCommentsDF = commentsDF.append(answersDF, ignore_index=True)
allCommentsDF.shape

(1433391, 3)

In [6]:
# вк собака объединяет символы - ковычки и троеточия
import emoji
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

punctuation = string.punctuation + '…«»—–'
def clean_text(text):
# Make text lowercase
    text = text.lower()
# remove text in square brackets
    text = re.sub(r'\[.*?\]', ' ', text)
# remove urls
    text = re.sub(r"http\S+", ' ', text)
# remove emojies
#     text = give_emoji_free_text(text)
# remove punctuation   
    text = re.sub(r'[%s]' % re.escape(punctuation), ' ', text)
# remove numbers
    text = re.sub(r'[0-9]', ' ', text)
# remove non letters
    text = re.sub(r'[^\w\s]', '', text)
# remove dublicate spaces
    text = re.sub('\s\s+', ' ', text)
    return text
comments_df_clean = pd.DataFrame(allCommentsDF.text.progress_apply(lambda x: clean_text(x)))

100%|██████████| 1433391/1433391 [00:55<00:00, 25985.99it/s]


In [7]:
%%time

stopwords_list = stopwords.words('russian') + stopwords.words('english')
stopwords_list.append('это')
stopwords_list.append('всё')
stopwords_list.append('ещё')
stopwords_list.append('весь')
stopwords_list.append('человек')
stopwords_list.append('свой')
stopwords_list.append('который')
stopwords_list.append('мочь')
stopwords_list.append('')

def lemmatize(text, lemmer = morph, stopwords = stopwords_list):
    words = text.split(' ')
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if len(w) >= 3 and not w in stopwords]

with Pool(processes = 4) as pool:
    lemmatized = pool.starmap(lemmatize, zip(comments_df_clean.text))

CPU times: user 10.4 s, sys: 9.48 s, total: 19.8 s
Wall time: 11min 49s


In [8]:
import json
with open('LDA/lemmatized.json', 'w') as f:
    f.write(json.dumps(lemmatized))

In [7]:
import json
with open('LDA/lemmatized.json', 'r') as f:
    lemmatized = json.loads(f.read())

In [8]:
comments_lemmatized_df = pd.DataFrame({'text' : lemmatized, 'date' : allCommentsDF.date})

In [9]:
def wordsCount(texts):
    count = Counter()
    for words in texts:
        for word in words:
            count[word] += 1
    return count

def removeNotCommonWords(text, word_counter):
    return [word for word in text if word_counter[word] > 10]

def func(df):
    word_counter = wordsCount(df.text.values.tolist())
    
    df.text = df.text.apply(lambda x: removeNotCommonWords(x, word_counter))
    df.text = df.text.apply(lambda x: ' '.join(x))
    df.text = df.text.apply(lambda x: x.split())
    df = df[df.text.apply(lambda x: len(x) >= 20 and len(x) <= 300)]
    return df

In [10]:
clean_texts = func(comments_lemmatized_df)

In [11]:
clean_texts

,text,date
4,"[действительно, движение, жизнь, ребёнок, вооб...",1616477087
9,"[очень, странный, рекомендация, ношение, маска...",1616511004
13,"[переболеть, ковид, многие, слабоумный, вести,...",1616425952
24,"[переболеть, ковид, многие, слабоумный, вести,...",1616426029
27,"[эксперт, воз, установить, животное, стать, пр...",1616481257
...,...,...
1433316,"[скулить, сказать, слово, значит, сразу, скули...",1579862018
1433332,"[обычный, тактика, фарма, компания, создать, н...",1579785429
1433358,"[белый, налёт, говорить, скорее, ангина, бакте...",1579700867
1433372,"[атрибут, единорос, переходящий, красный, знам...",1579638367


In [12]:
clean_texts['text_str'] = clean_texts.text.apply(lambda x: ' '.join(x))

In [15]:
clean_texts.text_str.to_csv('tmp_texts.txt', header=False, index=False)

# tone detection

In [11]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')  # [('всё', None), ('очень', None), ('плохо', None)]

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [19]:
messages = clean_texts.text_str.to_list()

In [21]:
clean_texts['tones'] = model.predict(messages, k=2)

In [10]:
import liwc
parse, category_names = liwc.load_token_parser('/home/mchelushkin/Downloads/Telegram Desktop/Russian_LIWC2007_Dictionary.dic')

In [61]:
clean_texts['sentiment_an'] = clean_texts.text.progress_apply(lambda x: fun_sentiment_analysis(x))

100%|██████████| 144155/144155 [00:30<00:00, 4796.38it/s]


In [62]:
clean_texts

,text,date,sentiment_an
4,"[действительно, движение, жизнь, ребёнок, вооб...",1616477087,"{'Действие': 0.06172839506172839, 'Позитив': 0..."
9,"[очень, странный, рекомендация, ношение, маска...",1616511004,"{'Функция': 0.08333333333333333, 'Наречие': 0...."
13,"[переболеть, ковид, многие, слабоумный, вести,...",1616425952,"{'Функция': 0.1, 'quant': 0.03333333333333333,..."
24,"[переболеть, ковид, многие, слабоумный, вести,...",1616426029,"{'Функция': 0.1, 'quant': 0.03333333333333333,..."
27,"[эксперт, воз, установить, животное, стать, пр...",1616481257,"{'Работа': 0.06451612903225806, 'Действие': 0...."
...,...,...,...
1433316,"[скулить, сказать, слово, значит, сразу, скули...",1579862018,"{'Общество': 0.0625, 'Перцепция': 0.0208333333..."
1433332,"[обычный, тактика, фарма, компания, создать, н...",1579785429,"{'Сравнение': 0.1694915254237288, 'Время': 0.1..."
1433358,"[белый, налёт, говорить, скорее, ангина, бакте...",1579700867,"{'Перцепция': 0.061224489795918366, 'Видение':..."
1433372,"[атрибут, единорос, переходящий, красный, знам...",1579638367,"{'Когнитив': 0.16, 'Мотивация': 0.04, 'Сравнен..."


In [57]:
def fun_sentiment_analysis(text):
    counter = dict(Counter(category for token in text for category in parse(token)))
    total = sum(counter.values())
    ans = []
    for x in counter:
        ans.append((x, counter[x] / total))
    return dict(ans)

## Подготовка данных для визуализации

In [6]:
def wordsCount(texts):
    count = Counter()
    for words in texts:
        for word in words:
            count[word] += 1
    return count

def removeNotCommonWords(text, word_counter):
    return [word for word in text if word_counter[word] > 10]

def func1(df):
    word_counter = wordsCount(df.text.values.tolist())
    
    df.text = df.text.apply(lambda x: removeNotCommonWords(x, word_counter))
    df.text = df.text.apply(lambda x: ' '.join(x))
    df.text = df.text.apply(lambda x: x.split())
#     df = df[df.text.apply(lambda x: len(x) >= 3)]
    df = df[df.text.apply(lambda x: len(x) >= 20 and len(x) <= 300)]
    return df

clean_texts_for_visualizarion = func1(comments_lemmatized_df)

In [25]:
clean_texts_for_visualizarion = clean_texts.copy()

In [26]:
other_corpus = [dictionary.doc2bow(text) for text in clean_texts_for_visualizarion.text]

In [27]:
topic_weights = []
topics = []
for row_list in tqdm(lda[other_corpus]):
    topic_weights.append([w for i, w in row_list])
    topics.append([i for i, w in row_list])

100%|██████████| 144155/144155 [00:55<00:00, 2589.89it/s]


In [28]:
topic_labels = list(range(1, 26))

clean_texts_for_visualizarion['topic_weights'] = topic_weights
clean_texts_for_visualizarion['topics'] = topics
clean_texts_for_visualizarion['best_topic'] = clean_texts_for_visualizarion.topic_weights.apply(lambda x: np.argmax(x))
clean_texts_for_visualizarion['formated_date'] = clean_texts_for_visualizarion.date.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')[:10])
best_topic_weights = []
best_topic_label = []
for row in clean_texts_for_visualizarion.iterrows():
    best_topic_weights.append(row[1]['topic_weights'][row[1].best_topic])
    best_topic_label.append(topic_labels[row[1]['topics'][row[1].best_topic]])

clean_texts_for_visualizarion['best_topic_weights'] = best_topic_weights
clean_texts_for_visualizarion['best_topic_label'] = best_topic_label

In [29]:
clean_texts_for_visualizarion['text_str'] = clean_texts_for_visualizarion.text.apply(lambda x: ' '.join(x))
clean_texts_for_visualizarion = clean_texts_for_visualizarion.drop_duplicates(subset=['text_str'])
confident_data = clean_texts_for_visualizarion[clean_texts_for_visualizarion.best_topic_weights >= 0.6]

In [77]:
clean_texts_for_visualizarion

,text,date,text_str,tones,topic_weights,topics,best_topic,formated_date,best_topic_weights,best_topic_label
4,"[действительно, движение, жизнь, ребёнок, вооб...",1616477087,действительно движение жизнь ребёнок вообще от...,"{'neutral': 0.4765896201133728, 'negative': 0....","[0.28972316, 0.12688696, 0.19843104, 0.2094424...","[3, 7, 9, 10, 15, 16, 21]",0,2021-03-23,0.289723,4
9,"[очень, странный, рекомендация, ношение, маска...",1616511004,очень странный рекомендация ношение маска реко...,"{'neutral': 0.5544804334640503, 'positive': 0....","[0.087134674, 0.5738357, 0.057179853, 0.2584905]","[1, 3, 4, 22]",1,2021-03-23,0.573836,4
13,"[переболеть, ковид, многие, слабоумный, вести,...",1616425952,переболеть ковид многие слабоумный вести редко...,"{'neutral': 0.5544804334640503, 'negative': 0....","[0.21666904, 0.0973471, 0.34014884, 0.31076983]","[0, 13, 22, 23]",2,2021-03-22,0.340149,23
27,"[эксперт, воз, установить, животное, стать, пр...",1616481257,эксперт воз установить животное стать промежут...,"{'neutral': 0.59267657995224, 'negative': 0.34...","[0.22344148, 0.20510943, 0.35604066, 0.18423465]","[4, 7, 8, 13]",2,2021-03-23,0.356041,9
30,"[учёный, заявить, смертельный, угроза, попадан...",1616343505,учёный заявить смертельный угроза попадание ко...,"{'neutral': 0.546748161315918, 'negative': 0.3...","[0.051789545, 0.25748527, 0.055870418, 0.15689...","[2, 4, 5, 8, 17, 18, 22]",1,2021-03-21,0.257485,5
...,...,...,...,...,...,...,...,...,...,...
1433316,"[скулить, сказать, слово, значит, сразу, скули...",1579862018,скулить сказать слово значит сразу скулить наз...,"{'neutral': 0.5000100135803223, 'negative': 0....","[0.2960938, 0.16482203, 0.024312858, 0.0451526...","[6, 7, 9, 18, 21, 23]",5,2020-01-24,0.296693,24
1433332,"[обычный, тактика, фарма, компания, создать, н...",1579785429,обычный тактика фарма компания создать новый в...,"{'neutral': 0.5312193632125854, 'negative': 0....","[0.06685541, 0.112264544, 0.050011493, 0.05655...","[4, 6, 8, 10, 17, 20]",4,2020-01-23,0.352559,18
1433358,"[белый, налёт, говорить, скорее, ангина, бакте...",1579700867,белый налёт говорить скорее ангина бактериальн...,"{'negative': 0.3629792034626007, 'neutral': 0....","[0.5435269, 0.0609086, 0.07339673, 0.03941582,...","[0, 4, 8, 15, 16, 23]",0,2020-01-22,0.543527,1
1433372,"[атрибут, единорос, переходящий, красный, знам...",1579638367,атрибут единорос переходящий красный знамя пос...,"{'neutral': 0.33459946513175964, 'negative': 0...","[0.23778933, 0.045923296, 0.28131568, 0.204614...","[7, 11, 14, 20, 22, 24]",2,2020-01-21,0.281316,15


In [78]:
clean_texts_for_visualizarion[['date', 'formated_date', 'best_topic_weights', 'best_topic_label', 'tones']].to_csv('data_for_visualization_with_tones_v1.csv', index_label='id')